<a href="https://colab.research.google.com/github/deanhoperobertson/Masters-/blob/master/Thesis/Code/Bi_LSTM_CRF_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bi-LSTM-CRF Model

In [1]:
from google.colab import files
src = list(files.upload().values())[0]
open('mylib.py','wb').write(src)
import mylib

Saving prepro.py to prepro (3).py


In [2]:
!sudo pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-l00pm82j
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-l00pm82j
  Stored in directory: /tmp/pip-ephem-wheel-cache-vxfaveny/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [73]:
import pandas as pd
import numpy as np
import urllib.request
from sklearn.model_selection import train_test_split

#cusotm packages
from prepro import readfile, readstring

#keras and tensorflow packages
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss


#evaluation
from sklearn_crfsuite.metrics import flat_classification_report,flat_f1_score,flat_precision_score
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

ModuleNotFoundError: ignored

In [0]:
#import data from my github repo
train_url = "https://raw.githubusercontent.com/deanhoperobertson/Masters-/master/Thesis/Code/Data/train.txt"
test_url = "https://raw.githubusercontent.com/deanhoperobertson/Masters-/master/Thesis/Code/Data/test.txt"
train = urllib.request.urlopen(train_url).read()
test = urllib.request.urlopen(test_url).read()
train = train.decode('utf-8')
test = test.decode('utf-8')

#preproces the txt file
train = readstring(train)
test = readstring(test)

#create corpus
corpus = train.copy()
corpus.extend(test)

In [30]:
words = []
tags = []
for sentence in corpus:
    for word in sentence:
        words.append(word[0])
        tags.append(word[1])
        
words=list(set(words))
n_words = len(words)
print("Number of words in the dataset: ", n_words)
tags = list(set(tags))
n_tags = len(tags)
print("Number of Labels: ", n_tags)

Number of words in the dataset:  27316
Number of Labels:  9


In [31]:
word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx["UNK"] = 1 # Unknown words
word2idx["PAD"] = 0 # Padding
# Vocabulary Key:token_index -> Value:word
idx2word = {i: w for w, i in word2idx.items()}
print("The word 'rejects' is identified by the index: {}".format(word2idx["rejects"]))

The word 'rejects' is identified by the index: 23125


In [32]:
# The first entry is reserved for PAD
tag2idx = {t: i+1 for i, t in enumerate(tags)}
tag2idx["PAD"] = 0
# Vocabulary Key:tag_index -> Value:Label/Tag
idx2tag = {i: w for w, i in tag2idx.items()}
print("The labels B-LOC (location) is identified by the index: {}".format(tag2idx["B-LOC"]))

The labels B-LOC (location) is identified by the index: 5


In [33]:
#Find the maxium length of the all the sentences in the corpus
length = []
for sentence in corpus:
  length.append(len(sentence))

MAX_LEN= max(length)
print("The maxium length of sentence is:",max(length))

The maxium length of sentence is: 124


In [34]:
# Convert each sentence from list of Token to list of word_index
X = [[word2idx[w[0]] for w in s] for s in train]

# Padding each sentence to have the same lenght
X = pad_sequences(maxlen=MAX_LEN, sequences=X, padding="post", value=word2idx["PAD"])
X

array([[15551, 23125,  5783, ...,     0,     0,     0],
       [ 8685, 21134,     0, ...,     0,     0,     0],
       [ 8493, 24726,     0, ...,     0,     0,     0],
       ...,
       [18853, 16569,   623, ...,     0,     0,     0],
       [25094, 26633,     0, ...,     0,     0,     0],
       [24086, 13319,  4815, ...,     0,     0,     0]], dtype=int32)

In [35]:
# Convert Tag/Label to tag_index
y = [[tag2idx[w[1]] for w in s] for s in train]

# Padding each sentence to have the same lenght
y = pad_sequences(maxlen=MAX_LEN, sequences=y, padding="post", value=tag2idx["PAD"])
y

array([[7, 9, 4, ..., 0, 0, 0],
       [1, 3, 0, ..., 0, 0, 0],
       [5, 9, 0, ..., 0, 0, 0],
       ...,
       [7, 9, 7, ..., 0, 0, 0],
       [9, 9, 0, ..., 0, 0, 0],
       [7, 9, 7, ..., 0, 0, 0]], dtype=int32)

In [0]:

# One-Hot encode categorical labels
y = [to_categorical(i, num_classes=n_tags+1) for i in y]

In [38]:
#split into test and train subsets
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)
X_tr.shape,np.array(y_tr).shape, X_te.shape, np.array(y_te).shape

((12636, 124), (12636, 124, 10), (1405, 124), (1405, 124, 10))

## Building LSTM-CRF Model

In [0]:
# Model definition
EMBEDDING=40

input = Input(shape=(MAX_LEN,))
model = Embedding(input_dim=n_words+2, output_dim=EMBEDDING, # n_words + 2 (PAD & UNK)
                  input_length=MAX_LEN, mask_zero=True)(input)

model = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model)

model = TimeDistributed(Dense(10, activation="relu"))(model)
crf = CRF(n_tags+1)  # CRF layer
out = crf(model)  # output
model = Model(input, out)

In [77]:
model.compile(optimizer="rmsprop", loss=crf_loss)#, metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 124)               0         
_________________________________________________________________
embedding_9 (Embedding)      (None, 124, 40)           1092720   
_________________________________________________________________
bidirectional_9 (Bidirection (None, 124, 100)          36400     
_________________________________________________________________
time_distributed_10 (TimeDis (None, 124, 10)           1010      
_________________________________________________________________
crf_9 (CRF)                  (None, 124, 10)           230       
Total params: 1,130,360
Trainable params: 1,130,360
Non-trainable params: 0
_________________________________________________________________


In [78]:
%%time
BATCH_SIZE = 500
EPOCHS=10

history = model.fit(X_tr, np.array(y_tr), batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.1, verbose=1)

Train on 11372 samples, validate on 1264 samples
Epoch 1/10
11372/11372 [==============================] - 18s 2ms/step - loss: 42.5971 - val_loss: 41.0894
Epoch 2/10
11372/11372 [==============================] - 14s 1ms/step - loss: 41.9364 - val_loss: 40.8593
Epoch 3/10
11372/11372 [==============================] - 14s 1ms/step - loss: 41.7545 - val_loss: 40.7223
Epoch 4/10
11372/11372 [==============================] - 14s 1ms/step - loss: 41.6230 - val_loss: 40.6037
Epoch 5/10
11372/11372 [==============================] - 14s 1ms/step - loss: 41.4975 - val_loss: 40.4805
Epoch 6/10
11372/11372 [==============================] - 14s 1ms/step - loss: 41.3721 - val_loss: 40.3774
Epoch 7/10
11372/11372 [==============================] - 14s 1ms/step - loss: 41.2750 - val_loss: 40.3053
Epoch 8/10
11372/11372 [==============================] - 14s 1ms/step - loss: 41.1998 - val_loss: 40.2514
Epoch 9/10
11372/11372 [==============================] - 14s 1ms/step - loss: 41.1368 - val_lo

## Evaluation

In [56]:
pred = model.predict(X_tr, verbose=1)

12636/12636 [==============================] - 77s 6ms/step


In [79]:
# TRain Eval
pred_cat = model.predict(X_tr)
pred = np.argmax(pred_cat, axis=-1)
y_tr_true = np.argmax(y_tr, -1)

# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in pred]
y_tr_true_tag = [[idx2tag[i] for i in row] for row in y_tr_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag)
print(report)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

       B-LOC       0.71      0.81      0.76      6440
      B-MISC       0.83      0.36      0.50      3108
       B-ORG       0.53      0.70      0.60      5672
       B-PER       0.63      0.71      0.67      5993
       I-LOC       0.00      0.00      0.00      1028
      I-MISC       0.58      0.01      0.03      1056
       I-ORG       0.69      0.21      0.32      3334
       I-PER       0.62      0.88      0.73      4118
           O       0.99      1.00      0.99    152588
         PAD       1.00      1.00      1.00   1383527

    accuracy                           0.99   1566864
   macro avg       0.66      0.57      0.56   1566864
weighted avg       0.99      0.99      0.99   1566864



In [80]:
X_pred = model.predict(X_tr, verbose=1)
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(X_pred)
test_labels = pred2label(y_tr)

12636/12636 [==============================] - 97s 8ms/step


In [81]:
print(flat_classification_report(test_labels, pred_labels))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

       B-LOC       0.71      0.81      0.76      6440
      B-MISC       0.83      0.36      0.50      3108
       B-ORG       0.53      0.70      0.60      5672
       B-PER       0.63      0.71      0.67      5993
       I-LOC       0.00      0.00      0.00      1028
      I-MISC       0.58      0.01      0.03      1056
       I-ORG       0.69      0.21      0.32      3334
       I-PER       0.62      0.88      0.73      4118
           O       1.00      1.00      1.00   1536115

    accuracy                           0.99   1566864
   macro avg       0.62      0.52      0.51   1566864
weighted avg       0.99      0.99      0.99   1566864



In [83]:
#F1 Score
flat_f1_score(test_labels, pred_labels,average='weighted')

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.9911701050301305